# 层和块

In [18]:
import torch
from torch import nn
from torch.nn import functional as F

In [19]:
# 多层感知机
""" nn.Sequential 定义了一个特殊的 Module"""
net = nn.Sequential(
    nn.Linear(20, 256), 
    nn.ReLU(), 
    nn.Linear(256, 10)
    )

X = torch.rand(size=(2,20), requires_grad=True)
net(X)

tensor([[ 0.0170, -0.1487, -0.2131, -0.5660, -0.3181, -0.3801, -0.1051,  0.0324,
         -0.0014, -0.0204],
        [-0.0153, -0.0353, -0.2139, -0.4092, -0.3671, -0.2834, -0.1464,  0.0227,
          0.0082,  0.0724]], grad_fn=<AddmmBackward0>)

任何层以及神经网络都是 Module 的子类

In [20]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

net = MLP()
net(X)

tensor([[ 0.1856,  0.0043,  0.2111, -0.1339, -0.1064, -0.1577, -0.0451, -0.0790,
          0.0848,  0.2023],
        [ 0.2732, -0.0770,  0.1067,  0.0679, -0.1388, -0.2292,  0.0382, -0.1848,
          0.0928,  0.1425]], grad_fn=<AddmmBackward0>)

In [21]:
""" 自定义Sequential """
class My_Sequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block    # block 有序字典
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = My_Sequential(
    nn.Linear(20, 256), 
    nn.ReLU(), 
    nn.Linear(256, 10)
    )

X = torch.rand(size=(2,20), requires_grad=True)
net(X)

tensor([[-0.1889,  0.0636, -0.0640,  0.1260, -0.2819, -0.1551,  0.0773,  0.0797,
          0.0096, -0.0399],
        [-0.1198,  0.1782,  0.0275,  0.1361, -0.1305, -0.0812,  0.0048,  0.0488,
         -0.0102,  0.0547]], grad_fn=<AddmmBackward0>)

## 正向传播中自定义计算

In [22]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)    # 随便放的值，不参与学习

    def forward(self, X):
        return sum(X) # 稀奇古怪的操作

net = FixedHiddenMLP()
net([1, 2])

3

In [23]:
""" 混合搭配 """
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20))
chimera(X)

tensor([[-0.1778, -0.1484, -0.0290, -0.1229,  0.2215,  0.0035, -0.1987, -0.1153,
          0.2252,  0.0297, -0.1748, -0.1443, -0.1022, -0.1991, -0.3512, -0.1368,
          0.0104,  0.1535, -0.1914, -0.1309],
        [-0.1632, -0.1407,  0.0022, -0.1309,  0.2488,  0.0107, -0.1924, -0.0948,
          0.1747,  0.0008, -0.1732, -0.1503, -0.1040, -0.2507, -0.3355, -0.1547,
          0.0335,  0.1668, -0.1651, -0.1523]], grad_fn=<AddmmBackward0>)